In [1]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models

import numpy as np 
import matplotlib.pyplot as plt

import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import Normalizer 
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import datetime, os
from skimage import data, color
from tensorflow.keras.layers import BatchNormalization
import tensorflow_datasets as tfds

print(tf.version)


<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.8/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [3]:
from os.path import dirname, join as pjoin
import scipy.io as sio


In [4]:
normal=  sio.loadmat('/content/drive/MyDrive/NormalRPeaks.mat')
af=  sio.loadmat('/content/drive/MyDrive/AfRPeaks.mat')

In [ ]:
normalImages=np.empty([normal['NormalRPeaks'].shape[1],360 ,360])
afImages=np.empty([af['AfRPeaks'].shape[1], 360,360])

normalImagesCropped=np.empty([normal['NormalRPeaks'].shape[1],112 ,112])
afImagesCropped=np.empty([af['AfRPeaks'].shape[1], 112,112])


In [ ]:

for i in range(0,normal['NormalRPeaks'].shape[1]):
 print("i")
 print(i)
#  x=np.diff(np.diff(normal['NormalRPeaks'][0][i][0]))
 x=np.diff(np.array(normal['NormalRPeaks'][0][i][0],dtype=np.int16),2)
 fig = plt.figure(figsize=(5,5))
 plt.plot(x[0:-2],x[1:-1])
 plt.xlim([-500, 500])
 plt.ylim([-500, 500])
 plt.axis('off')
# plt.show()
 fig.canvas.draw()       # draw the canvas, cache the renderer

# width, height = fig.get_size_inches() * fig.get_dpi()
# mplimage = np.fromstring(fig.canvas.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
# print(image.shape)

 data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
 image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
 image = image[:,:,0]/256
 normalImages[i,:,:]=image
 




In [ ]:
np.save("normalImages.npy", normalImages)

In [ ]:
def imageCropper(samples, REQUIRED_SIZE: int): # Returns samples with required size
    from skimage.transform import resize
    cropCount = -1
    PADDING_VAL = samples[0][0][0]
    ORIGINAL_SIZE = samples.shape[1]

    for sample in samples:
        sampleMaxCrop = -1
        for row in range(ORIGINAL_SIZE):
            rowMaxCrop = 0
            for col in range(ORIGINAL_SIZE):
                if sample[row][col] == sample[row][ORIGINAL_SIZE - 1 - col] == PADDING_VAL:
                    rowMaxCrop += 1
            if rowMaxCrop < sampleMaxCrop or sampleMaxCrop == -1:
                sampleMaxCrop = rowMaxCrop
        if sampleMaxCrop < cropCount or cropCount == -1:
            cropCount = sampleMaxCrop

        for col in range(ORIGINAL_SIZE):
            colMaxCrop = 0
            for row in range(ORIGINAL_SIZE):
                if sample[row][col] == sample[ORIGINAL_SIZE - 1 - row][col] == PADDING_VAL:
                    colMaxCrop += 1
            if colMaxCrop < sampleMaxCrop:
                sampleMaxCrop = colMaxCrop
        if sampleMaxCrop < cropCount:
            cropCount = sampleMaxCrop

    print('initial size: ', samples.shape)
    cropCount = min(cropCount, int((ORIGINAL_SIZE - REQUIRED_SIZE) / 2))
    if cropCount > 0:
        samples = samples[:, cropCount:-cropCount, cropCount:-cropCount]
        print('after cropping: ', samples.shape)
    if samples.shape[1] > REQUIRED_SIZE:
        import numpy as np
        shrinked_arr = np.empty(
            shape=(samples.shape[0], REQUIRED_SIZE, REQUIRED_SIZE))
        for i in range(samples.shape[0]):
            shrinked_arr[i] = resize(
                samples[i], (REQUIRED_SIZE, REQUIRED_SIZE))
        print('after interpolation: ', shrinked_arr.shape)
        return shrinked_arr
    return samples

In [ ]:
for i in range(0,af['AfRPeaks'].shape[1]):
 print("i")
 print(i)
 i_new=i
 x=np.diff(np.array(af['AfRPeaks'][0][i_new][0],dtype=np.int16),2)
 fig = plt.figure(figsize=(5,5))
 plt.plot(x[0:-2],x[1:-1])
 plt.xlim([-500, 500])
 plt.ylim([-500, 500])
 plt.axis('off')
 fig.canvas.draw()       # draw the canvas, cache the renderer


 data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
 image = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
 image = image[:,:,0]/256
 afImages[i,:,:]=image
 
 print(image.shape)

In [ ]:
np.save("afImages.npy", afImages)

In [ ]:
normalImages=np.empty([normal['NormalRPeaks'].shape[1],360 ,360])
afImages=np.empty([af['AfRPeaks'].shape[1], 360,360])

normalImagesCropped=np.empty([normal['NormalRPeaks'].shape[1],112 ,112])
afImagesCropped=np.empty([af['AfRPeaks'].shape[1], 112,112])


In [ ]:
normalImagesCropped = imageCropper(normalImages, 112)
np.save("normalImagesCropped.npy", normalImagesCropped)

In [ ]:
afImagesCropped = imageCropper(afImages, 112)
np.save("afImagesCropped.npy", afImagesCropped)